![](https://raw.githubusercontent.com/wandb/wandb/508982e50e82c54cbf0dd464a9959fee0e1740ad/.github/wb-logo-lightbg.png)
<!--- @wandbcode{dataval-course-03} -->

# Schema Validation

In this notebook, we will implement TFX's schema validation to see if any of the corruptions in the previous notebook were detected. We'll log the results of schema validation to wandb.

You can set up wandb alerts here: https://docs.wandb.ai/guides/runs/alert

I use Modal because TFDV doesn't run on Mac M1s. You can create a free account on Modal here: https://modal.com/signup -- it comes with $10/month credits, which should be plenty more than enough to run the notebooks in this course. Once you have created an account, follow the "Getting Started" instructions on the homepage:

* Run `pip install modal-client` (also included in `requirements.txt` in this repo)
* Run `modal token new`, which will open a browser window and authenticate you with your account

Then you should be able to run this notebook!

In [ ]:
from dataval.dataset import WeatherDataset
from dataval.train import CatBoostTrainer
from dataval import dataset_extensions

import os
import matplotlib.pyplot as plt
import modal
import pandas as pd

import wandb
from wandb import AlertLevel

In [ ]:
image = (
    modal.Image.debian_slim()
    .pip_install_from_requirements("requirements.txt")
    .pip_install(["tensorflow-data-validation", "tensorflow_metadata", "protobuf==3.20.0"])
)
stub = modal.Stub("tfdv-tutorial", image=image)

In [ ]:
# Load dataset

ds = WeatherDataset(os.path.join(os.getcwd(), "canonical-paritioned-dataset"), sample_frac=0.2)

In [ ]:
train_df = ds.load(ds.get_partition_keys()[0])
test_df = ds.load(ds.get_partition_keys()[1])

In [ ]:
train_df

## Infer schema and check test data for errors

From the train dataframe, we create a schema using TFDV. Then we use this schema to find anomalies in the test data. We apply this to the original dataframes first.

In [ ]:
@stub.function()
def find_anomalies(train_df, test_df):
    import tensorflow_data_validation as tfdv
    from google.protobuf.json_format import MessageToDict
    
    train_stats =  tfdv.generate_statistics_from_dataframe(train_df)
    schema = tfdv.infer_schema(statistics=train_stats)
    test_stats = tfdv.generate_statistics_from_dataframe(test_df)
    
    anomalies = tfdv.validate_statistics(statistics=test_stats, schema=schema)
    anomalies_df = tfdv.utils.display_util.get_anomalies_dataframe(anomalies)
    # return MessageToDict(anomalies)
    return anomalies_df

In [ ]:
with stub.run():
    X_train, _ = ds.split_feature_label(train_df)
    X_test, _ = ds.split_feature_label(test_df)
    anomalies = find_anomalies.call(X_train, X_test)

In [ ]:
anomalies

Seems like the raw data did not have any anomalies!

## Iterate through corruptions

See if tfdv detects any anomalies, for all the corruptions we had in our previous notebook.

In [ ]:
X_train, _ = ds.split_feature_label(train_df)
corruption_anomalies = {}

for corruption_name, corruption_res in ds.iterate_corruptions_by_feature(test_df, "cmc", corruption_rate=0.05):
    corrupted_test_df, corrupted_columns = corruption_res
    corrupted_X_test, _ = ds.split_feature_label(corrupted_test_df)
    with stub.run():
        corruption_anomalies[corruption_name] = find_anomalies.call(X_train, corrupted_X_test)

In [ ]:
# Send wandb alerts

run = wandb.init(project="ml-dataval-tutorial", tags=["TFDV-schema"])

for corruption_name, anomalies in corruption_anomalies.items():
    if len(anomalies) > 0:
        table = wandb.Table(dataframe=anomalies)
        wandb.log({corruption_name: table})
    
        wandb.alert(
            title=f"Errors detected in {corruption_name} experiment", 
            text = f"Found {len(anomalies)} anomalies",
            level=AlertLevel.WARN,
        )
        print(f"Found {len(anomalies)} in {corruption_name} experiment")

wandb.finish()

## Takeaways

It looks like only the `corrupt_null` corruption was flagged by schema validation! Maybe other validation techniques might flag them. Nevertheless, all the corruptions that schema validation found were true corruptions, so there isn't a false positive alert problem here.